In [1]:

import json
import pandas as pd
from pathlib import Path


In [2]:
# Defining paths
group_data = Path("/Users/harshul/website clone/harshul/test/group-data/")

MEMBERS_DIR_PATH = group_data / 'members/'
WEBSITE_DATA_PATH = group_data / 'website_data/'
CONTENT_DIR_PATH = WEBSITE_DATA_PATH / 'content'


In [3]:
#pd.DataFrame(experiences +education)
# if MSU in insti and no end_date

In [4]:
def extract_member_data(member_dir):
    if member_dir.name.startswith('.'):
        return None

    member_record = json.load(open(member_dir / 'info.json'))
    member_json_dir = member_dir / 'jsons'


    if (member_experiences := (member_json_dir / 'experiences.json')).exists():
        experiences = json.load(open(member_experiences))
    else:
        experiences = []


    if (member_education := (member_json_dir / 'education.json')).exists():
        education = json.load(open(member_education))
    else:
        education = []


    return member_record, experiences, education
def create_education_experience_df(experiences, education):
    ex_edu_df = pd.DataFrame(experiences + education)
    return ex_edu_df


def create_education_experience_df(experiences, education):
    ex_edu_df = pd.DataFrame(experiences + education)
   
    return ex_edu_df

import pandas as pd


def process_education_experience(ex_edu_df):
    filtered_df = pd.DataFrame()
    if 'institution' in ex_edu_df.columns and 'role' in ex_edu_df.columns and 'end_date' in ex_edu_df.columns:
        filtered_df = ex_edu_df[(ex_edu_df['institution'] == 'Michigan State University') & 
                                ex_edu_df['role'].notna() & 
                                ex_edu_df['end_date'].isna()]
        return filtered_df
    else:
        return pd.DataFrame()


def assign_academic_role(df):
    if df.empty:
        return 'Other'

    top_record = df.iloc[0]

    if  top_record.get('role', '') == 'Assistant Professor' or top_record.get('role', '')=='Professor':
        return 'Professor'
    if top_record.get('role', '') in ['Visiting Researcher', 'Postdoctoral Researcher']:
        return 'Postdoctoral researcher'
    if top_record.get('degree', '') == 'Bachelors':
        return 'Graduate Student' if top_record.get('end_date') else 'Undergraduate Student'
    if top_record.get('degree', '') in ['Masters', 'PhD']:
        return 'Graduate Student'
    else:
        return 'Other'

member_records = []
education_experience_dfs = []

for member_dir in MEMBERS_DIR_PATH.glob('*'):
    if member_dir.name.startswith('.'):
        continue

    member_record, experiences, education = extract_member_data(member_dir)

    education_experience_df = create_education_experience_df(experiences, education)
    education_experience_df = process_education_experience(education_experience_df)
    academic_role = assign_academic_role(education_experience_df)

    nick_name = member_record.get('nick_name', '')
    first_name = member_record.get('first_name', '')
    last_name = member_record.get('last_name', '')

    name = f"{nick_name or first_name} {last_name}"

    member_records.append({"Name": name, "Academic Role": academic_role})
    education_experience_dfs.append(education_experience_df)

people_page_df = pd.DataFrame(member_records)


education_experience_df = pd.concat(education_experience_dfs, ignore_index=True)
education_experience_df


,institution,city,state,country,role,start_date,end_date,degree,subject,start-date
0,Michigan State University,East Lansing,Michigan,USA,Visiting Researcher,2023-09-01,NaN,NaN,NaN,NaN
1,Michigan State University,East Lansing,Michigan,USA,Research Consultant at ICER,2022-01-01,NaN,NaN,NaN,NaN
2,Michigan State University,East Lansing,Michigan,USA,Postdoctoral Researcher,2023-07-10,NaN,NaN,NaN,NaN
3,Michigan State University,East Lansing,Michigan,US,Assistant Professor,2019-08-15,NaN,NaN,NaN,NaN
